In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
%matplotlib notebook
import math

class FoB_raw:
    def __init__(self):
        self.alledata = None
        
    def laad(self, bestandsnaam):
        # Laad raw data van Flock of Birds
        # Controle op inhoud ontbreekt, dus bij ontbrekende data bv. ontstaan mogelijk fouten.
    
        myfile = open(bestandsnaam, "r")
        self.alledata = None   # Matrix met alle data van myfile

        # verzamel steeds 4 regels uit de file. Deze 4 regels vormen samen 1 sample van 1 sensor
        a = [[],[],[],[]] # a[0] ..a[3] bevat alle waarden van deze 4 regels
        i = 0             # teller, bij welke regel zijn we

        for line in myfile:
            if len(line) > 1:               # sla lege regels over (regel kan \n of \r bevatten)
                for n in line.split():      # converteer elk 'woord' naar een float
                    a[i].append(float(n))
                i = i + 1                   # regel gelezen
        
            # voeg 4 regels van de input samen tot 1 rij in alldata
            if (i == 4):    
                #tmp = [ a[0][0], a[0][1:4], [a[1],a[2],a[3]]]
                #print(tmp)
                if self.alledata is None:
                    self.alledata = np.transpose(a[0]+a[1]+a[2]+a[3])
                else:
                    self.alledata = np.vstack([self.alledata, np.transpose(a[0]+a[1]+a[2]+a[3])])
                i = 0
                a = [[],[],[],[]]

    def sensornummers(self):
        # Geef een array met de unieke sensornummers
        return np.unique(self.alledata[:,0])
    
    def aantalSamples(self):
        # Geef het aantal samples dat per sensor in de data staat
        lengte = self.sensornummers()
        return int(self.alledata.shape[0] / lengte.size)
    
    def sample(self, nummer):
        # Geef een bepaald sample van de data.
        # Geeft een array met alle data van het sample. Het sample bevat alle sensoren gesorteerd op sensornummer, 
        start = nummer * self.sensornummers().size
        end = start + self.sensornummers().size
        tmp = self.alledata[start:end]
        tmp = tmp[tmp[:,0].argsort()]
        return tmp
    
    def sensordata(self, sensor):
        # Geef alle data van één bepaalde sensor
        # in: de index, waar in de array 'sensornummers' de sensor staat.
        #     Dus geef 0 op om de waarde van de eerste sensor te krijgen, ook als die sensor gelabeld is met bv '2'
        resultaat = self.sample(0)[sensor]
        for i in range(1, int(self.aantalSamples())):
            resultaat = np.vstack([resultaat, self.sample(i)[sensor]])
        return resultaat    
                
#of zijn classes te lastig??
def laadRawData(bestandsnaam):
    # Laad raw data van Flock of Birds
    # Controle op inhoud ontbreekt, dus bij ontbrekende data bv. ontstaan mogelijk fouten.
    
    myfile = open(bestandsnaam, "r")
    alldata = None   # Matrix met alle data van myfile

    # verzamel steeds 4 regels uit de file. Deze 4 regels vormen samen 1 sample van 1 sensor
    a = [[],[],[],[]] # a[0] ..a[3] bevat alle waarden van deze 4 regels
    i = 0             # teller, bij welke regel zijn we

    for line in myfile:
        if len(line) > 1:               # sla lege regels over (regel kan \n of \r bevatten)
            for n in line.split():      # converteer elk 'woord' naar een float
                a[i].append(float(n))
            i = i + 1                   # regel gelezen
        
        # voeg 4 regels van de input samen tot 1 rij in alldata
        if (i == 4):    
            #tmp = [ a[0][0], a[0][1:4], [a[1],a[2],a[3]]]
            #print(tmp)
            if alldata is None:
                alldata = np.transpose(a[0]+a[1]+a[2]+a[3])
            else:
                alldata = np.vstack([alldata, np.transpose(a[0]+a[1]+a[2]+a[3])])
            i = 0
            a = [[],[],[],[]]
    return alldata

In [2]:
AB1 = FoB_raw()
AB1.laad("AB1.txt")

In [3]:
def Ellebooghoek(Schouder, Elleboog, Pols):
    # P1 & P2 & P3 are lists, filled like: [x,y,z]
    
    def CalculateLength(Point1, Point2):
        # Point2 & Point2 are lists, filled like: [x,y,z]
        # https://math.stackexchange.com/questions/42640/calculate-distance-in-3d-space
        XLength = abs(Point1[0] - Point2[0])
        YLength = abs(Point1[1] - Point2[1])
        ZLength = abs(Point1[2] - Point2[2])
        Distance = math.sqrt((XLength ** 2) + (YLength ** 2) + (ZLength ** 2))
        return Distance
    
    def SSSTriangle(LengthA, LengthB, LengthC):
        # Will return Only angle of A
        # LengthA will be the line  opposite of corner A
        # LengthB will be the line  opposite of corner B
        # LengthC will be the line  opposite of corner C
        # https://www.mathsisfun.com/algebra/trig-solving-sss-triangles.html
        CosAngleA = ((LengthB ** 2) + (LengthC ** 2) - (LengthA ** 2)) / (2 * LengthB * LengthC)
        AngleA = math.acos(CosAngleA)
        return AngleA
    
    BovenArm = CalculateLength(Schouder, Elleboog)
    OnderArm = CalculateLength(Elleboog, Pols)
    NepArm = CalculateLength(Schouder, Pols) # Laatste lengthe voor de SSS Driehoek
    Ellebooghoek = SSSTriangle(NepArm, BovenArm, OnderArm)
    return math.degrees(Ellebooghoek)
# hoek = Ellebooghoek(Schouder, Elleboog, Pols)
# print(hoek)

In [4]:
def update_lines(num):
    x = AB1.sample(num)
    plt.clf()
    ax = fig.add_subplot(111, projection='3d')
    Thorax = [x[1][xas], x[1][yas], x[1][zas]]
    
    LSchouder = [x[2][xas], x[2][yas], x[2][zas]]
    LElleboog = [x[3][xas], x[3][yas], x[3][zas]]
    LPols = [x[4][xas], x[4][yas], x[4][zas]]
    
    RSchouder = [x[5][xas], x[5][yas], x[5][zas]]
    RElleboog = [x[6][xas], x[6][yas], x[6][zas]]
    RPols = [x[7][xas], x[7][yas], x[7][zas]]

    # Ellebooghoeken berekenen
    LElleboogHoek = Ellebooghoek(LSchouder, LElleboog, LPols)
    RElleboogHoek = Ellebooghoek(RSchouder, RElleboog, RPols)
    
    # Arm hoeken
    LArmHoek = Ellebooghoek(LSchouder, Thorax, LElleboog)
    RArmHoek = Ellebooghoek(RSchouder, Thorax, RElleboog)
    ax.text(800, -800, 500,  '%s' % ('R Olecranon: ' + str(round(RElleboogHoek, roundnum)) + \
                                     '\nL Olecranon: ' + str(round(LElleboogHoek, roundnum)) + \
                                     '\n \nR Armhoek: ' + str(round(RArmHoek, roundnum)) + \
                                     '\nL Armhoek: ' + str(round(LArmHoek, roundnum))), size=textsize*0.75, zorder=1, color='k')
    
    # Middle stuff + Base sensor
    ax.scatter(x[0][xas], x[0][yas], x[0][zas] * -1, c='r')
    ax.text(x[0][xas],x[0][yas],x[0][zas] * -1,  '%s' % (int(AB1.sensornummers()[0])), size=textsize, zorder=1, color='k') 
    ax.scatter(x[1][xas], x[1][yas], x[1][zas] * -1, c='b')
    ax.text(x[1][xas], x[1][yas], x[1][zas] * -1,  '%s' % (int(AB1.sensornummers()[1])), size=textsize, zorder=1, color='k')
    
    # Medical Left side
    ax.scatter(x[2][xas], x[2][yas], x[2][zas] * -1, c='cyan')
    ax.text(x[2][xas], x[2][yas], x[2][zas] * -1,  '%s' % (int(AB1.sensornummers()[2])), size=textsize, zorder=1, color='k')
    ax.plot([x[1][xas], x[2][xas]], [x[1][yas], x[2][yas]], [x[1][zas] * -1, x[2][zas] * -1], c='b')
    ax.scatter(x[3][xas], x[3][yas], x[3][zas] * -1, c='c')
    ax.text(x[3][xas], x[3][yas], x[3][zas] * -1,  '%s' % (int(AB1.sensornummers()[3])), size=textsize, zorder=1, color='k')
    ax.plot([x[3][xas], x[2][xas]], [x[3][yas], x[2][yas]], [x[3][zas] * -1, x[2][zas] * -1], c='b')
    ax.scatter(x[4][xas], x[4][yas], x[4][zas] * -1, c='darkcyan')
    ax.text(x[4][xas], x[4][yas], x[4][zas] * -1,  '%s' % (int(AB1.sensornummers()[4])), size=textsize, zorder=1, color='k')
    ax.plot([x[3][xas], x[4][xas]], [x[3][yas], x[4][yas]], [x[3][zas] * -1, x[4][zas] * -1], c='b')

    # Medical Recht side
    ax.scatter(x[5][xas], x[5][yas], x[5][zas] * -1, c='magenta')
    ax.text(x[5][xas], x[5][yas], x[5][zas] * -1,  '%s' % (int(AB1.sensornummers()[5])), size=textsize, zorder=1, color='k')
    ax.plot([x[1][xas], x[5][xas]], [x[1][yas], x[5][yas]], [x[1][zas] * -1, x[5][zas] * -1], c='m')
    ax.scatter(x[6][xas], x[6][yas], x[6][zas] * -1, c='m')
    ax.text(x[6][xas], x[6][yas], x[6][zas] * -1,  '%s' % (int(AB1.sensornummers()[6])), size=textsize, zorder=1, color='k')
    ax.plot([x[6][xas], x[5][xas]], [x[6][yas], x[5][yas]], [x[6][zas] * -1, x[5][zas] * -1], c='m')
    ax.scatter(x[7][xas], x[7][yas], x[7][zas] * -1, c='darkmagenta')
    ax.text(x[7][xas], x[7][yas], x[7][zas] * -1,  '%s' % (int(AB1.sensornummers()[7])), size=textsize, zorder=1, color='k')
    ax.plot([x[6][xas], x[7][xas]], [x[6][yas], x[7][yas]], [x[6][zas] * -1, x[7][zas] * -1], c='m')
    ax.set_zlim(-400,600)
    ax.set_ylim(-800,800)
    ax.set_xlim(800,1500)
    return ax

In [9]:
"""
============
3D animation
============

Made by the best programmer of the group!
"""
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
# import seaborn

textsize = 12

# Define what columns is the X, Y and Z axis
xas = 1
yas = 2
zas = 3
roundnum = 2

fig = plt.figure()

ax = update_lines(0)
# print(AB1.sample(0))
# print(ax[0]._verts3d)



# Creating the Animation object
frames = AB1.aantalSamples()

print('aantal frames:%s' % frames)
line_ani = animation.FuncAnimation(fig, update_lines, frames,
                                   interval=126)

# Writer = animation.writers['ffmpeg']
# writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

# line_ani.save('lines.mp4', writer=writer)
# line_ani.save('movie.mp4', writer='ffmpeg', codec='rawvideo')
# line_ani.save('movie2.mp4')
# line_ani.save('im.mp4', writer=writer)
# plt.legend()
plt.show()

aantal frames:96


ValueError: Image size of 2281231x1255701 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 432x288 with 1 Axes>